In [1]:
import pandas as pd
import numpy as np

history = pd.read_csv('../data/sbb-data.csv', sep=';')
lignes = pd.read_csv('../data/sbb-ligne.csv', sep=';')

In [2]:
import json

ligne = lignes[lignes["ÉCARTEMENT"] == 'N']
ligne = ligne.reset_index()
t = ligne["Geo Shape"]
t = t.apply(json.loads)

for i in range(0, len(t)):
    t[i] = t[i]["coordinates"]

t = [[ [t[i][j][1], t[i][j][0]] for j in range(0, len(t[i]))] for i in range(0, len(t))]

In [3]:
history = history.dropna(subset=['Geopos'])
history = history.groupby(['Stop name', 'Geopos']).size().reset_index(name='Count')
history["Geopos"] = history["Geopos"].str.split(",")

In [17]:
import folium

m = folium.Map(location=[46.8, 8.2], zoom_start=8, tiles="Cartodb Positron")
# MarkerCluster(IR90_5_57["Geopos"]).add_to(m)
for i in range(0, len(t)):
    folium.PolyLine(t[i], color="#050505", weight=2.5, opacity=1).add_to(m)

for i in range(len(history)):
    folium.CircleMarker(location=history["Geopos"][i], popup=history["Stop name"][i] + " - " + str(history["Count"][i]), radius=(history["Count"][i]/100), fill=True,fill_opacity=1, color="red").add_to(m)
m.save('../test.html')

In [4]:
import folium
from folium.plugins import MarkerCluster

# Création de la carte
m = folium.Map(location=[46.8, 8.2], zoom_start=8, tiles="Cartodb Positron")

# Création des groupes de fonctionnalités pour les lignes et les gares
lignes = folium.FeatureGroup(name="Lignes")
gares = folium.FeatureGroup(name="Gares")

# Ajout des lignes avec PolyLine au groupe 'lignes'
for i in range(len(t)):
    folium.PolyLine(t[i], color="#050505", weight=2.5, opacity=1).add_to(lignes)

# Ajout des marqueurs de gares avec CircleMarker au groupe 'gares'
for i in range(len(history)):
    folium.CircleMarker(
        location=history["Geopos"][i],
        popup=history["Stop name"][i] + " - " + str(history["Count"][i]),
        radius=(history["Count"][i] / 100),
        fill=True,
        fill_opacity=1,
        color="red"
    ).add_to(gares)

# Ajout des groupes de fonctionnalités à la carte
lignes.add_to(m)
gares.add_to(m)

# Ajout d'un contrôle de couche pour afficher/masquer les lignes et les gares
folium.LayerControl(collapsed=False).add_to(m)

# Enregistrement de la carte dans un fichier HTML
m.save('../test2.html')
